In [54]:
import tensorflow as tf

TRAIN_PATH = '/home/jupyter/bwt_data/transformed/train/*.csv'
EVAL_PATH  = '/home/jupyter/bwt_data/transformed/eval/*.csv'
TEST_PATH  = '/home/jupyter/bwt_data/transformed/test/*.csv'
LABEL_COLUMN = "weight_pounds"

def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=5, # Artificially small to make examples easier to show.
        label_name=LABEL_COLUMN,
        na_value="UKN",
        num_epochs=1,
        ignore_errors=True, 
        **kwargs
    )
    return dataset

train_data = get_dataset(TRAIN_PATH)
eval_data  = get_dataset(EVAL_PATH)
test_data  = get_dataset(TEST_PATH)


In [55]:
categorical_columns = []
categorical_columns.append(
    tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_vocabulary_list('mother_race', vocabulary_list=['White', 'Black', 'American Indian', 'Chinese', 'Japanese', 'Hawaiian', 'Filipino', 'Asian Indian', 'Korean', 'Samaon', 'Vietnamese', 'Unkown'])
    )
)
categorical_columns.append(
    tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_vocabulary_list("plurality", vocabulary_list=[1,2,3])
    )
)

numeric_columns = []
numeric_columns.append(
    tf.feature_column.numeric_column("mother_age")
)
numeric_columns.append(
    tf.feature_column.numeric_column("gestation_weeks")
)

preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [ ]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(1, kernel_initializer='normal')
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])

model.fit(train_data,
          validation_data=eval_data,
          epochs=5)

test_metrics = model.evaluate(test_data)
print("Test Performance: MAE = {}, MSE = {}".format(*test_metrics[1:]))

Epoch 1/5
20000/20000 [==============================] - 317s 16ms/step - loss: 1.3966 - mae: 0.9009 - mse: 1.3966 - val_loss: 0.0000e+00 - val_mae: 0.0000e+00 - val_mse: 0.0000e+00
Epoch 2/5
20000/20000 [==============================] - 248s 12ms/step - loss: 1.2919 - mae: 0.8730 - mse: 1.2918 - val_loss: 1.1715 - val_mae: 0.8371 - val_mse: 1.1715
Epoch 3/5
20000/20000 [==============================] - 238s 12ms/step - loss: 1.2569 - mae: 0.8635 - mse: 1.2569 - val_loss: 1.1420 - val_mae: 0.8302 - val_mse: 1.1421
Epoch 4/5
20000/20000 [==============================] - 244s 12ms/step - loss: 1.2386 - mae: 0.8598 - mse: 1.2387 - val_loss: 1.3832 - val_mae: 0.9184 - val_mse: 1.3832
Epoch 5/5
20000/20000 [==============================] - 235s 12ms/step - loss: 1.2261 - mae: 0.8559 - mse: 1.2261 - val_loss: 1.1219 - val_mae: 0.8224 - val_mse: 1.1219
   5883/Unknown - 47s 8ms/step - loss: 1.1047 - mae: 0.8134 - mse: 1.1047- 44s 8ms/step -  - 44s 8ms/step - loss: 1.104 - 45s 8ms/step - l